# Classifyinng snd-cert
This notebook aims to get the best optimal value of n and r for the negative selection algorithm in the snd-umn dataset.

In [ ]:
experiment_folder = "snd-unm"

In [ ]:
import pandas as pd
from data_treatment import merge_datasets
from model import main
from validation import calculate_roc_auc_components
from viz import plot_best_aucs_per_n, plot_auc_optimization, plot_roc_auc_from_file

In [ ]:
merge_datasets(
    [f"syscalls/{experiment_folder}/{experiment_folder}.1.test", f"syscalls/{experiment_folder}/{experiment_folder}.2.test", f"syscalls/{experiment_folder}/{experiment_folder}.3.test"],
    f"syscalls/{experiment_folder}/{experiment_folder}.test"
)

merge_datasets(
    [f"syscalls/{experiment_folder}/{experiment_folder}.1.labels", f"syscalls/{experiment_folder}/{experiment_folder}.2.labels", f"syscalls/{experiment_folder}/{experiment_folder}.3.labels"],
    f"syscalls/{experiment_folder}/{experiment_folder}.labels"
)

In [ ]:
auc_metric = {}

for n in [7, 8, 9, 10, 11, 12]:
    auc_metric[n] = {}
    for r in [2, 3, 4, 5]:
        main(
            f"syscalls/{experiment_folder}/{experiment_folder}.train",
            f"syscalls/{experiment_folder}/{experiment_folder}.test",
            n,
            r,
            f"syscalls/{experiment_folder}/{experiment_folder}.labels",
            f"syscalls/{experiment_folder}/results.csv",
        )
        
        results = pd.read_csv(f"syscalls/{experiment_folder}/results.csv")

        auc_me, _, _ = calculate_roc_auc_components(results)
        auc_metric[n][r] = auc_me
        print(f"{n} {r} {auc_metric[n][r]}")
    
    plot_auc_optimization(
        n,
        [2, 3, 4, 5],
        [auc_metric[n][2], auc_metric[n][3], auc_metric[n][4], auc_metric[n][5]],
    )

In [ ]:
best_r_values = []

best_n = 0
best_r_value = 0

for n in [7, 8, 9, 10, 11, 12]:
    # find the best r for each n
    best_r = max(auc_metric[n], key=auc_metric[n].get)
    best_r_values.append(auc_metric[n][best_r])

    if auc_metric[n][best_r] > best_r_value:
        best_n = n
        best_r_value = auc_metric[n][best_r]
    

plot_best_aucs_per_n(
    [7, 8, 9, 10, 11, 12],
    best_r_values,
)

In [ ]:
for r in [2, 3, 4, 5]:
    main(
        f"syscalls/{experiment_folder}/{experiment_folder}.train",
        f"syscalls/{experiment_folder}/{experiment_folder}.test",
        n,
        r,
        f"syscalls/{experiment_folder}/{experiment_folder}.labels",
        f"syscalls/{experiment_folder}/results.csv",
    )
    

    plot_roc_auc_from_file(
        f"syscalls/{experiment_folder}/results.csv",
        best_n,
        r
    )